In [1]:
# -*- coding: utf-8 -*-

import pandas as pd
import geohash
import sqlite3 as sql
import datetime
import sys
import sqlalchemy
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from itertools import *
from pandas import ExcelWriter

# Use chinese for MPL
mpl.rcParams['font.family'] = 'Hiragino Sans GB'
mpl.rcParams['font.sans-serif'] = 'Hiragino Sans GB'
mpl.rcParams['text.latex.unicode']=True

In [2]:
database_path = 'sqlite3/'
databases = {'2015-08-18':'2015-08-18-data.db',
             '2015-08-29':'2015-08-29-data.db',
             '2015-09-29':'2015-09-29-data.db',
             '2015-10-02':'2015-10-02-data.db',
             '2015-10-30':'2015-10-30-data.db',
             '2016-01-08':'2016-01-08-data.db',
             '2016-02-01':'2016-02-01-data.db'}

DISK_CATEGORY_KEYWORDS = {
    'staple': ['面', '饭', '粥', '馒头', '花卷', '馄饨', '饺', '包', '粉', '饼'],
    'drinking': ['酒,''咖啡', '雪碧', '可乐', '茶', '拿铁'],
    'dessert': ['布丁', '蛋糕', '饼干', '曲奇']
}

MINOR_CATEGORY_TEXT = {
    208: '品牌快餐',
    209: '盖浇饭',
    210: '中式炒菜',
    211: '披萨意面',
    212: '汉堡',
    213: '米粉面馆',
    214: '麻辣烫',
    215: '包子粥店',
    216: '生煎锅贴',
    217: '饺子馄饨',
    218: '烧烤',
    219: '香锅',
    221: '川湘菜',
    222: '粤菜',
    223: '东北菜',
    224: '云南菜',
    225: '江浙菜',
    226: '西北菜',
    227: '鲁菜',
    228: '清真',
    229: '日韩料理',
    230: '西餐',
    231: '火锅',
    232: '海鲜',
    234: '炸鸡炸串',
    235: '鸭脖卤味',
    236: '小龙虾',
    237: '地方小吃',
    238: '零食',
    240: '饮品',
    241: '甜品',
    242: '咖啡',
    243: '点心',
    249: '蛋糕',
    250: '面包'
}


def determine_dish_type(name):
    for type, keywords in DISK_CATEGORY_KEYWORDS.items():
        for keyword in keywords:
            if keyword in name:
                return type
    return 'vegetable'

def load_database(db_name):
    print('Loading database:', db_name, '...')
    engine = sqlalchemy.create_engine('sqlite:///' + database_path + db_name)
#     print('加载表: restaurants')
    restaurants_db = pd.read_sql_table('restaurants', engine)
#     print('加载表: menus')
    menus_db = pd.read_sql_table('menus', engine)
    
    category_db = pd.read_sql_table('category', engine)
    restaurant_categories_db = pd.read_sql_table('restaurant_categories', engine)
    
#     print('计算营业额...')
    menus_db['revenue'] = menus_db['price'] * menus_db['month_sales']
    
#     print('合并营业额...')
    revenue_db = menus_db.loc[:, ['restaurant_id', 'revenue']].groupby(
                'restaurant_id').sum().reset_index(drop=False)
    restaurants_db = pd.merge(restaurants_db, revenue_db, left_on='id', right_on='restaurant_id',
                                       how='left')
    
#     print('计算菜单平均价...')
    mean_db = menus_db.loc[:, ['restaurant_id', 'price']].groupby('restaurant_id').mean().reset_index(
            drop=False).rename(columns={'price': 'mean_price'})
    restaurants_db = pd.merge(restaurants_db, mean_db, on='restaurant_id')
    
#     print('计算平均价格...')
    restaurants_db['average_price'] = restaurants_db['revenue'] / restaurants_db['month_sales']

    restaurants_db['revenue'] = restaurants_db['revenue'].fillna(0)
    del restaurants_db['restaurant_id']

#     print('合并商家类型...')
    restaurants_db = pd.merge(restaurants_db, restaurant_categories_db, left_on='id',
                                   right_on='restaurant_id', how='right')
    del restaurants_db['restaurant_id']

#     print('合并类型详细信息...')
    category_db = category_db.rename(columns={'id': 'cat_id', 'name': 'cat_name'})
    restaurants_db = pd.merge(restaurants_db, category_db, left_on='category_id', right_on='cat_id',
                                   how='left')
    del restaurants_db['category_id']
    
    
#     print('为菜单生成种类分类信息...')
    menus_db['type'] = [determine_dish_type(n) for n in menus_db['name']]

#     print('为菜单生成商铺分类信息...')
    category_db = restaurants_db.loc[:, ['id', 'cat_name']].rename(columns={'id': 'restaurant_id'})
    menus_db = pd.merge(menus_db, category_db, on='restaurant_id')    

    print('database:',db_name,'loaded!')
    return restaurants_db,menus_db

In [3]:
dft = load_database('2016-02-01-data.db')

Loading database: 2016-02-01-data.db ...
database: 2016-02-01-data.db loaded!


# Summary

In [40]:
rest_db = dft[0]

union_db = rest_db.drop_duplicates('id')



count_of_restaurants = union_db.shape[0]
print('采样商家数量: {:,} 户'.format(count_of_restaurants))

total_revenue = union_db['revenue'].sum()
print('总营业额: {:,.2f} 元'.format(total_revenue))

total_sales = union_db['month_sales'].sum()
print('总销量: {:,} 份'.format(total_sales))

average_revenue = total_revenue / total_sales
print('平均每单营业额: {:,.2f} 元'.format(average_revenue))

highest_revenue = union_db.sort_values(by='revenue', ascending=False).iloc[0]['revenue']
print('单月最高营业额: {:,.2f}'.format(highest_revenue))

采样商家数量: 11,856 户
总营业额: 415,170,486.87 元
总销量: 28,525,839.0 份
平均每单营业额: 14.55 元
单月最高营业额: 1,372,474.20


In [79]:
# 计算营收排行前10名
revenues = {}
revenues['category'] = []
revenues['revenue'] = []
revenues['sales'] = []
revenues['num_restaurants'] = []



for category in MINOR_CATEGORY_TEXT.values():
    df = rest_db[rest_db.cat_name == category]
    total_revenue = df['revenue'].sum()
    total_sales = df['month_sales'].sum()
    revenues['category'].append(category)
    revenues['revenue'].append(total_revenue)
    revenues['sales'].append(total_sales)
    revenues['num_restaurants'].append(df.shape[0])
    
df_ranking = pd.DataFrame(revenues)
df_ranking['average_per_sale'] = df_ranking['revenue'] / df_ranking['sales']
df_ranking['average_per_restaurant'] = df_ranking['revenue'] / df_ranking['num_restaurants']


with ExcelWriter('numbers_material.xlsx') as writer:
    print('营收额前10类')

    sorted_df = df_ranking.sort_values(by='revenue', ascending=False).reset_index(drop=True).iloc[0:10]
    ax = sorted_df.plot(kind='bar',x='category',y = 'revenue')
    ax.set_xlabel('分类')
    ax.set_ylabel('营业额')
    plt.show()
    
    sorted_df.to_excel(writer,'revenue_ranking')

    print('平均每单价格最高10类')

    sorted_df = df_ranking.sort_values(by='average_per_sale', ascending=False).reset_index(drop=True).iloc[0:10]
    ax = sorted_df.plot(kind='bar',x='category',y = 'average_per_sale')
    ax.set_xlabel('分类')
    ax.set_ylabel('平均每单价格')
    plt.show()

    sorted_df.to_excel(writer,'average_per_sale')

    print('平均每商家营业额最高10类')

    sorted_df = df_ranking.sort_values(by='average_per_restaurant', ascending=False).reset_index(drop=True).iloc[0:10]
    ax = sorted_df.plot(kind='bar',x='category',y = 'average_per_restaurant')
    ax.set_xlabel('分类')
    ax.set_ylabel('平均每商家营业额')
    plt.show()
    
    sorted_df.to_excel(writer,'average_per_restaurant')



营收额前10类
平均每单价格最高10类
平均每商家营业额最高10类
